In [1]:
from tpot import TPOTClassifier
import pandas as pd
import sklearn.model_selection

/home/vzb403/.local/lib/python3.10/site-packages/tpot/builtins/__init__.py:36: UserWarning: Warning: optional dependency `torch` is not available. - skipping import of NN models.
  warnings.warn("Warning: optional dependency `torch` is not available. - skipping import of NN models.")


In [ ]:
%pip install torch

In [2]:
df_master = pd.read_parquet('../master/master.parquet')

In [3]:
for colToStd in ['webCount', 'webQuestionableUsage','numExtEmails','sumExtEmailAttachments','sumExtEmailSize']:
    df_roleStd = df_master.groupby('role').apply(lambda x: (x[colToStd] - x[colToStd].mean())/x[colToStd].std()).to_frame()\
    .reset_index(level=0).drop('role', axis=1).rename(columns={colToStd:colToStd+'RoleStd'})
    df_master = df_master.join(df_roleStd)

In [4]:
for colToStd in ['webCount', 'webQuestionableUsage','numExtEmails','sumExtEmailAttachments','sumExtEmailSize']:
    df_roleStd = df_master.groupby('user_id').apply(lambda x: (x[colToStd] - x[colToStd].mean())/x[colToStd].std()).to_frame()\
    .reset_index(level=0).drop('user_id', axis=1).rename(columns={colToStd:colToStd+'UserStd'})
    df_master = df_master.join(df_roleStd)

In [5]:
df_master = df_master.fillna(0)
y=df_master['isInsider']
X = df_master.drop(['isInsider','user_id', 'dateString', 'userDate', 'dateHour','employee_name','role'], axis=1)

In [6]:
X.columns

Index(['isEmployed', 'hasLogin', 'firstActivity', 'hasConnect',
       'probLogonRole', 'webCount', 'countPC', 'psyc_O', 'psyc_C', 'psyc_E',
       'psyc_A', 'psyc_N', 'numMonthsEmployed', 'sumExtEmailSize',
       'sumExtEmailAttachments', 'numExtEmails', 'probLogonUser',
       'webQuestionableUsage', 'webCountRoleStd',
       'webQuestionableUsageRoleStd', 'numExtEmailsRoleStd',
       'sumExtEmailAttachmentsRoleStd', 'sumExtEmailSizeRoleStd',
       'webCountUserStd', 'webQuestionableUsageUserStd', 'numExtEmailsUserStd',
       'sumExtEmailAttachmentsUserStd', 'sumExtEmailSizeUserStd'],
      dtype='object')

In [9]:
X.iloc[45455]

isEmployed                       1.000000e+00
hasLogin                         2.000000e+00
firstActivity                    0.000000e+00
hasConnect                       0.000000e+00
probLogonRole                    4.081745e-02
webCount                         0.000000e+00
countPC                          2.000000e+00
psyc_O                           2.200000e+01
psyc_C                           3.300000e+01
psyc_E                           2.000000e+01
psyc_A                           1.300000e+01
psyc_N                           2.500000e+01
numMonthsEmployed                5.000000e+00
sumExtEmailSize                  0.000000e+00
sumExtEmailAttachments           0.000000e+00
numExtEmails                     0.000000e+00
probLogonUser                    6.947473e-07
webQuestionableUsage             0.000000e+00
webCountRoleStd                  0.000000e+00
webQuestionableUsageRoleStd      0.000000e+00
numExtEmailsRoleStd              0.000000e+00
sumExtEmailAttachmentsRoleStd    0

In [7]:
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y, random_state=42, stratify=y)

In [10]:
tpot = TPOTClassifier(generations=5, 
                      population_size=50, 
                      verbosity=2,
                      cv=5,
                      n_jobs=-1,
                      scoring='f1_micro')

tpot.fit(X_train, y_train)

Optimization Progress:   0%|          | 0/300 [00:00<?, ?pipeline/s]

/home/vzb403/.local/lib/python3.10/site-packages/tpot/builtins/__init__.py:36: UserWarning: Warning: optional dependency `torch` is not available. - skipping import of NN models.
  warnings.warn("Warning: optional dependency `torch` is not available. - skipping import of NN models.")
/home/vzb403/.local/lib/python3.10/site-packages/tpot/builtins/__init__.py:36: UserWarning: Warning: optional dependency `torch` is not available. - skipping import of NN models.
  warnings.warn("Warning: optional dependency `torch` is not available. - skipping import of NN models.")
/home/vzb403/.local/lib/python3.10/site-packages/tpot/builtins/__init__.py:36: UserWarning: Warning: optional dependency `torch` is not available. - skipping import of NN models.
  warnings.warn("Warning: optional dependency `torch` is not available. - skipping import of NN models.")
/home/vzb403/.local/lib/python3.10/site-packages/tpot/builtins/__init__.py:36: UserWarning: Warning: optional dependency `torch` is not available

In [ ]:
for i, individual in enumerate(sorted(tpot.evaluated_individuals_)):
    print(individual)